<a href="https://colab.research.google.com/github/nravinuthala/EVA4.0/blob/master/EVA4_Session4Nagaraj_17_1k_BN_Dropout_99_4_11thEpoch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()   
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)   
        self.batchnorm1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)  
        self.batchnorm2 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)       
        self.dropout1 = nn.Dropout(0.25)        
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1) 
        self.batchnorm3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 8, 3, padding=1)
        self.batchnorm4 = nn.BatchNorm2d(8)
        self.pool2 = nn.MaxPool2d(2, 2)   
        self.dropout2 = nn.Dropout(0.25)                    
        self.conv5 = nn.Conv2d(8, 16, 3) 
        self.batchnorm5 = nn.BatchNorm2d(16)          
        self.conv6 = nn.Conv2d(16, 32, 3)  
        self.batchnorm6 = nn.BatchNorm2d(32)  
        self.dropout3 = nn.Dropout(0.25)              
        self.conv7 = nn.Conv2d(32, 10, 3)

    def forward(self, x):
        x = self.dropout1(self.pool1(self.batchnorm2(F.relu(self.conv2(self.batchnorm1(F.relu(self.conv1(x))))))))
        x = self.dropout2(self.pool2(self.batchnorm4(F.relu(self.conv4(self.batchnorm3(F.relu(self.conv3(x))))))))
        x = self.dropout3(self.batchnorm6(F.relu(self.conv6(self.batchnorm5(F.relu(self.conv5(x)))))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 28, 28]           1,168
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
           Dropout-6           [-1, 16, 14, 14]               0
            Conv2d-7           [-1, 32, 14, 14]           4,640
       BatchNorm2d-8           [-1, 32, 14, 14]              64
            Conv2d-9            [-1, 8, 14, 14]           2,312
      BatchNorm2d-10            [-1, 8, 14, 14]              16
        MaxPool2d-11              [-1, 8, 7, 7]               0
          Dropout-12              [-1, 8, 7, 7]               0
           Conv2d-13             [-1, 16, 5, 5]           1,168
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128 # defining batch size

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {} #defining dynamic keyword arguments as number of args would be diff for cuda and for cpu
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, #load predefined mnist dataset designated for training
                    transform=transforms.Compose([        #composing multiple transforms together
                        transforms.ToTensor(),          #transfor to create tensor from input data
                        transforms.Normalize((0.1307,), (0.3081,))  #transform to normalize data with dataset's mean and std
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(            #same as above for testing set
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train() #set the model to train
    pbar = tqdm(train_loader) #progress bar to show training progress
    for batch_idx, (data, target) in enumerate(pbar): #loop over each batch
        data, target = data.to(device), target.to(device) # load data and target into device
        #data = data.view(batch_size, data.size(0))
        #target = target.view(batch_size)
        optimizer.zero_grad() #zero out gradients
        output = model(data)  #apply model to data -- forward pass
        loss = F.nll_loss(output, target) #compute loss
        loss.backward() #backward propagation of loss
        optimizer.step() #adjust params to minimize loss
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}') #update progress bar with loss details


def test(model, device, test_loader):
    model.eval() #evaluate the model trained in previous step
    test_loss = 0
    correct = 0
    with torch.no_grad(): 
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            #target = target.view(batch_size)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(0, 19):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0519, Accuracy: 9827/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0367, Accuracy: 9872/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0179, Accuracy: 9941/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0188, Accuracy: 9941/10000 (99%)



loss=0.022607853636145592 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.02it/s]



Test set: Average loss: 0.0174, Accuracy: 9945/10000 (99%)

